<a href="https://colab.research.google.com/github/SL-LAIDLAW/projet-outilBioinfo/blob/master/Projet_Outils_Bioinfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet de HMSN204
_Sean LAIDLAW_, _Mathieu Blaison_

### TODO: diagramme de Gantt qui fera une synth`ese des tˆaches effectu´ees et de l’implication de chaque ´etudiant.

Liste des taches et dates (pour faire Gantt après)


*  creation du document colab &  planification du projet = mercredi 27 mars
*  requete NCBI pour sequences sex1 = Samedi, 06 avril  2019 + lundi 08/apr



## Traitements Biopython de SEX1

### Obtention des sequences sauvages et mutés, du NCBI

In [0]:
!pip install biopython
from Bio import SeqIO 
from Bio import Entrez

In [0]:
Entrez.email = "sean.laidlaw@etu.umontpellier.fr"
search_string = "SEX1[Gene] AND Arabidopsis thaliana[Organism]"
req = Entrez.esearch(db= "nucleotide", term=search_string)  # ID of fasta of SEX1 gene
res = Entrez.read(req)
res

In [0]:
sex1_id = res["IdList"]
req.close()
seqgb  = Entrez.efetch (db="nucleotide",id=sex1_id, rettype='fasta')
seqselect = SeqIO.parse(seqgb, "fasta")

sex1_variants = []
for sequence in seqselect:
    sex1_variants.append(sequence.seq)
sex1_variants

[Seq('CACAGATACCGTCCATCAACGTATCACTGGGCTCTCTTTCTCCGTGCCGTTCCT...ATT', SingleLetterAlphabet()),
 Seq('TATCTTCTGCAATTCGTGGTTTGTGAACACAAGAGAAAAAAAAAAAAAAACATA...AAG', SingleLetterAlphabet()),
 Seq('TATCTTCTGCAATTCGTGGTTTGTGAACACAAGAGAAAAAAAAAAAAAAACATA...TGG', SingleLetterAlphabet()),
 Seq('CCCTAAACCCTAAACCCTAAACCCTAAACCTCTGAATCCTTAATCCCTAAATCC...GGG', SingleLetterAlphabet()),
 Seq('CCCTAAACCCTAAACCCTAAACCCTAAACCTCTGAATCCTTAATCCCTAAATCC...GGG', SingleLetterAlphabet())]

In [0]:
def AlignSeqs(seqA, seqB):
    seqAlen = len(seqA)
    seqBlen = len(seqB)
    
    if seqA < seqB:
      smallest_seqlen = seqA
    else:
      smallest_seqlen = seqB
      
    seqA = seqA[:smallest_seqlen]
    seqB = seqB[:smallest_seqlen]
    
    df = pd.DataFrame()
    for i in range(0,smallest_seqlen):
      if seqA[i] == seqB[i]:
        df[i+1][i+1] = df[i][i]
      
AlignSeqs(sex1_variants[0].__str__() ,sex1_variants[1].__str__())

TypeError: ignored

## Alignement des séquences associées aux gènes sauvages/mutés

### un alignement par paire global des séquences d’ARNm

### donner les positions des variations sauvage/muté

In [0]:
myseq = myseqs[0]
myseq

Seq('CACAGATACCGTCCATCAACGTATCACTGGGCTCTCTTTCTCCGTGCCGTTCCT...ATT', SingleLetterAlphabet())

### taux de GC / traduction pour obtenir les séquences protéiques

récupation séquence (à modifier)



In [0]:
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
# replace this with Entrez
coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG", IUPAC.unambiguous_dna)

In [0]:
from Bio.SeqUtils import GC
GC(myseq)

43.37823834196891

In [0]:
coding_dna.translate()

/usr/local/lib/python3.6/dist-packages/Bio/Seq.py:2609: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


Seq('HRYRPSTYHWALFLRAVPPTCSNNDVPRGGIEG*SVVFYFDYT*VYVCVSE*RR...NSN', HasStopCodon(ExtendedIUPACProtein(), '*'))

# Base de données

## Reflection sur le type de base de données

On a eu le choix d’un des SGBD vu ce semestre : chado/postgresql (module phenotype), relationnel seul (oracle, postgresqlou mysql), neo4j, couchdb, triplestore jena/rdf. 

On a choisi d'eviter les base de données NoSQL car nous avons plus d'experience avec ceux là. 

Notre choix s'est ainsi  porté sur la base de données Chado. Celle-ci est conçue spécialement pour gérer les représentations complexes de données biologiques.

## Modelisation de la base de données

## Implantation de la base de données et peuplement

In [0]:
from Bio import SeqIO
import pandas as pd

ModuleNotFoundError: ignored

In [0]:
#from google.colab import files
#uploaded = files.upload()
df = pd.read_csv("Data_Projet_Entete.csv",sep=";")
df.head()

NameError: ignored

In [0]:
cov = df.cov()
cov

,PR length,Tortuosity,RootArea,LR length,Grouping,Plants/group,distanceFromHypocotyl std (cm),ShootArea,Chlorophyll (a.u)
PR length,10.748196,0.028717,0.360917,14.306058,-5.536155,5.666499,2.713649,0.177008,3.564540
Tortuosity,0.028717,0.005974,0.001301,0.050465,-0.052913,0.022540,0.010562,0.000559,0.012890
RootArea,0.360917,0.001301,0.021034,0.777904,-0.281867,0.333221,0.134795,0.008780,0.173087
LR length,14.306058,0.050465,0.777904,34.435065,-9.316980,11.438061,5.365691,0.336841,6.175700
Grouping,-5.536155,-0.052913,-0.281867,-9.316980,17.675330,-6.068031,-1.994934,-0.133922,-2.935009
Plants/group,5.666499,0.022540,0.333221,11.438061,-6.068031,10.899694,2.463164,0.163863,3.474913
distanceFromHypocotyl std (cm),2.713649,0.010562,0.134795,5.365691,-1.994934,2.463164,1.161365,0.058837,1.158878
ShootArea,0.177008,0.000559,0.008780,0.336841,-0.133922,0.163863,0.058837,0.009249,0.179739
Chlorophyll (a.u),3.564540,0.012890,0.173087,6.175700,-2.935009,3.474913,1.158878,0.179739,3.828083
